# Trade Size Distributions and Befords Law

## Bored Ape Collection

In [212]:
import os
import pandas as pd
cwd = os.getcwd()
#Load in the csv
bored_ape_location = cwd + "/data/bored_ape.csv"
bored_ape = pd.read_csv(bored_ape_location, low_memory=False)
bored_ape_converted = bored_ape

In [214]:
bored_ape_converted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32314 entries, 0 to 32313
Columns: 1468 entries, Unnamed: 0 to asset_bundle_assets_16_owner_config
dtypes: float64(613), int64(1), object(854)
memory usage: 361.9+ MB


In [201]:
bored_ape_converted.dropna(subset=['total_price'], inplace=True)
bored_ape_converted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24449 entries, 126 to 32313
Columns: 1468 entries, Unnamed: 0 to asset_bundle_assets_16_owner_config
dtypes: float64(613), int64(1), object(854)
memory usage: 274.0+ MB


In [202]:
prices = pd.DataFrame(bored_ape_converted['total_price'])

In [203]:
prices.head(10)

,total_price
126,36880000000000000000
127,40000000000000000000
128,40000000000000000000
129,33460000000000000000
130,99000000000000000000
131,55000000000000000000
253,1000000000000000
254,42690000000000000000
255,43490000000000000000
256,38000000000000000000


In [204]:
prices.dtypes

total_price    object
dtype: object

In [205]:
def first_sig_fig(number):
    """Returns the first significant digit of a provided number as string
    
    Parameters
    ----------
    number: The number whose first significant digit will be returned
    
    Raises
    ------
    TypeError: If the provided variable is not a number a TypeError will be raised
    """
    #Check that what is provided is actually a number
    if type(number) != int and type(number) != float and isinstance(number, numpy) == False:
        raise TypeError(f"{number} is not a number, it is of type {type(number)}")
    #Turn number into string so that it's iterable
    snumber = str(number)
    #Sentinel value to determine if we've hit the decimals yet.
    decimal_encountered = False
    for i in range(0,len(snumber)):
        if snumber[i].isdigit():
            temp = snumber[i]
            if snumber[i] == '0':
                pass
            else:
                return snumber[i]
        else:
            pass

In [206]:
def benford(array):
    """Takes in an array of prices as int or float and returns 
    the distribution of the first significant digit as a dict of each 
    digit and the respective count.
    
    Parameters
    ----------
    array: The numpy array to work with
    """
    c1, c2, c3, c4, c5, c6, c7, c8, c9 = 0, 0, 0, 0, 0, 0, 0, 0, 0
    for value in array:
        if first_sig_fig(value) == '1':
            c1 += 1
        elif first_sig_fig(value) == '2':
            c2 += 1
        elif first_sig_fig(value) == '3':
            c3 += 1
        elif first_sig_fig(value) == '4':
            c4 += 1
        elif first_sig_fig(value) == '5':
            c5 += 1
        elif first_sig_fig(value) == '6':
            c6 += 1
        elif first_sig_fig(value) == '7':
            c7 += 1
        elif first_sig_fig(value) == '8':
            c8 += 1
        elif first_sig_fig(value) == '9':
            c9 += 1
        else:
            raise Exception(f"First sig fig is not a number 1-9")
    counts = {'1':c1, '2':c2, '3':c3, '4':c4, '5':c5, '6':c6, '7':c7, '8':c8, '9':c9}
    return counts

In [207]:
#For some reason, all of the prices are in float, and have a bunch of trailing zeros
#This lambda function removes all the trailing zeros
temp = bored_ape_converted['total_price'].map(lambda price: price.rstrip('0'))

In [208]:
temp
#type(bored_ape_converted['payment_token_usd_price'])
#bored_ape_converted['payment_token_usd_price'].info()
#finalprice = temp.mul(bored_ape_converted['payment_token_usd_price'])

126       3688
127          4
128          4
129       3346
130         99
         ...  
32309       64
32310     7688
32311     6469
32312       67
32313    12345
Name: total_price, Length: 24449, dtype: object

In [216]:
bored_ape_converted['payment_token_usd_price'].isna().sum()

7865

In [217]:
temp.isna().sum()

0